In [27]:
%%writefile task1_create_external_table.hql
-- Creating external table posts_sample_external

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)

ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");

Overwriting task1_create_external_table.hql


In [28]:
%%writefile task1_create_managed_table.hql
-- creating managed table with partitioned by year and month

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

Overwriting task1_create_managed_table.hql


In [34]:
%%writefile task1_insert_table.hql

-- Inserting data into the managed table

set hive.exec.max.dynamic.partitions=300;
set hive.exec.max.dynamic.partitions.pernode=256;

USE demodb;
-- filling managed posts table from external table
SET hive.exec.dynamic.partition.mode=nonstrict;

-- Insert query
FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, year, month
WHERE year IS NOT NULL and month is not NULL;

Overwriting task1_insert_table.hql


In [31]:
%%writefile task1_result.hql
-- Your code here

USE demodb;

SELECT concat_ws("\t", year, concat_ws("-", year, month), c)
FROM (
    SELECT year, month, STRING(COUNT(*)) as c, ROW_NUMBER() OVER (ORDER BY year, month) as rw
    FROM posts_sample
    GROUP BY year, month
) as tmp
WHERE rw=3;

Overwriting task1_result.hql


In [35]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [36]:
!cat task1.hql

-- Creating external table posts_sample_external

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)

ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");
-- creating managed table with partitioned by year and month

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

-- Inserting data into the managed table

set hive.exec.max.dynamic.partitions=300;
set hive.exec.max.dynamic.partitions.pernode=256;

USE dem

In [37]:
%%bash
hive -f task1.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.749 seconds
OK
Time taken: 0.296 seconds
OK
Time taken: 0.181 seconds
OK
Time taken: 0.036 seconds
OK
Time taken: 1.092 seconds
OK
Time taken: 0.158 seconds
OK
Time taken: 0.025 seconds
Query ID = jovyan_20210427123316_3529313e-4793-4dd1-bc32-5bee879f319a
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1619521145414_0014, Tracking URL = http://172.17.0.62:8088/proxy/application_1619521145414_0014/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kil